In [ ]:
"""
Simple SQL-Based Validation for FGDB Conversion
Uses only SQL Server queries to validate the conversion results
"""

import pyodbc
import pandas as pd

class SimpleSQLValidator:
    def __init__(self, sql_config):
        self.sql_config = sql_config
        self.conn = None

    def connect(self):
        """Create SQL Server connection"""
        try:
            conn_str = (
                f"DRIVER={{ODBC Driver 17 for SQL Server}};"
                f"SERVER={self.sql_config['server']};"
                f"DATABASE={self.sql_config['database']};"
                f"UID={self.sql_config['username']};"
                f"PWD={self.sql_config['password']};"
            )
            self.conn = pyodbc.connect(conn_str)
            print("✅ Connected to SQL Server")
            return True
        except Exception as e:
            print(f"❌ Connection failed: {e}")
            return False

    def run_query(self, query, description):
        """Execute a query and return results"""
        try:
            print(f"\n🔍 {description}")
            df = pd.read_sql(query, self.conn)
            return df
        except Exception as e:
            print(f"❌ Query failed: {e}")
            return None

    def validate_basic_structure(self):
        """Validate basic table structure and record counts"""
        print("\n" + "="*60)
        print("📋 BASIC STRUCTURE VALIDATION")
        print("="*60)

        # Check if tables exist and get record counts
        query = """
        SELECT
            t.TABLE_NAME,
            (SELECT COUNT(*) FROM states WHERE t.TABLE_NAME = 'states') AS RecordCount
        FROM INFORMATION_SCHEMA.TABLES t
        WHERE t.TABLE_NAME = 'states'

        UNION ALL

        SELECT
            t.TABLE_NAME,
            (SELECT COUNT(*) FROM fedlandp WHERE t.TABLE_NAME = 'fedlandp') AS RecordCount
        FROM INFORMATION_SCHEMA.TABLES t
        WHERE t.TABLE_NAME = 'fedlandp'

        UNION ALL

        SELECT
            t.TABLE_NAME,
            (SELECT COUNT(*) FROM park_dtl WHERE t.TABLE_NAME = 'park_dtl') AS RecordCount
        FROM INFORMATION_SCHEMA.TABLES t
        WHERE t.TABLE_NAME = 'park_dtl'

        UNION ALL

        SELECT
            t.TABLE_NAME,
            (SELECT COUNT(*) FROM dtl_st WHERE t.TABLE_NAME = 'dtl_st') AS RecordCount
        FROM INFORMATION_SCHEMA.TABLES t
        WHERE t.TABLE_NAME = 'dtl_st'

        ORDER BY TABLE_NAME
        """

        df = self.run_query(query, "Checking table existence and record counts")
        if df is not None:
            print("\n📊 Table Summary:")
            for _, row in df.iterrows():
                print(f"   ✅ {row['TABLE_NAME']}: {row['RecordCount']:,} records")

            total_records = df['RecordCount'].sum()
            print(f"\n📈 Total Records: {total_records:,}")

            # Expected counts from our conversion
            expected = {'states': 51, 'fedlandp': 3687, 'park_dtl': 26307, 'dtl_st': 51}
            print(f"\n🎯 Expected vs Actual:")
            for _, row in df.iterrows():
                table = row['TABLE_NAME']
                actual = row['RecordCount']
                exp = expected.get(table, 0)
                status = "✅" if actual == exp else "❌"
                print(f"   {status} {table}: {actual:,} (expected {exp:,})")

        return df

    def validate_geometry(self):
        """Validate geometry data quality"""
        print("\n" + "="*60)
        print("🗺️ GEOMETRY VALIDATION")
        print("="*60)

        query = """
        SELECT
            'states' AS TableName,
            COUNT(*) AS TotalRecords,
            COUNT(Shape) AS RecordsWithGeometry,
            SUM(CASE WHEN Shape IS NOT NULL AND Shape.STIsValid() = 1 THEN 1 ELSE 0 END) AS ValidGeometries,
            SUM(CASE WHEN Shape IS NOT NULL AND Shape.STIsValid() = 0 THEN 1 ELSE 0 END) AS InvalidGeometries,
            AVG(CAST(Shape.STArea() AS FLOAT)) AS AvgArea
        FROM states

        UNION ALL

        SELECT
            'fedlandp' AS TableName,
            COUNT(*) AS TotalRecords,
            COUNT(Shape) AS RecordsWithGeometry,
            SUM(CASE WHEN Shape IS NOT NULL AND Shape.STIsValid() = 1 THEN 1 ELSE 0 END) AS ValidGeometries,
            SUM(CASE WHEN Shape IS NOT NULL AND Shape.STIsValid() = 0 THEN 1 ELSE 0 END) AS InvalidGeometries,
            AVG(CAST(Shape.STArea() AS FLOAT)) AS AvgArea
        FROM fedlandp

        UNION ALL

        SELECT
            'park_dtl' AS TableName,
            COUNT(*) AS TotalRecords,
            COUNT(Shape) AS RecordsWithGeometry,
            SUM(CASE WHEN Shape IS NOT NULL AND Shape.STIsValid() = 1 THEN 1 ELSE 0 END) AS ValidGeometries,
            SUM(CASE WHEN Shape IS NOT NULL AND Shape.STIsValid() = 0 THEN 1 ELSE 0 END) AS InvalidGeometries,
            AVG(CAST(Shape.STArea() AS FLOAT)) AS AvgArea
        FROM park_dtl

        UNION ALL

        SELECT
            'dtl_st' AS TableName,
            COUNT(*) AS TotalRecords,
            COUNT(Shape) AS RecordsWithGeometry,
            SUM(CASE WHEN Shape IS NOT NULL AND Shape.STIsValid() = 1 THEN 1 ELSE 0 END) AS ValidGeometries,
            SUM(CASE WHEN Shape IS NOT NULL AND Shape.STIsValid() = 0 THEN 1 ELSE 0 END) AS InvalidGeometries,
            AVG(CAST(Shape.STArea() AS FLOAT)) AS AvgArea
        FROM dtl_st

        ORDER BY TableName
        """

        df = self.run_query(query, "Validating geometry data quality")
        if df is not None:
            print("\n🗺️ Geometry Summary:")
            for _, row in df.iterrows():
                table = row['TableName']
                total = row['TotalRecords']
                with_geom = row['RecordsWithGeometry']
                valid = row['ValidGeometries']
                invalid = row['InvalidGeometries']
                avg_area = row['AvgArea']

                print(f"\n   📋 {table}:")
                print(f"      Total Records: {total:,}")
                print(f"      With Geometry: {with_geom:,}")
                print(f"      Valid Geometry: {valid:,}")
                print(f"      Invalid Geometry: {invalid:,}")
                print(f"      Avg Area: {avg_area:,.2f} sq meters")

                # Validation status
                if total == with_geom == valid and invalid == 0:
                    print(f"      Status: ✅ PERFECT")
                elif invalid == 0:
                    print(f"      Status: ✅ GOOD")
                else:
                    print(f"      Status: ⚠️ HAS ISSUES")

        return df

    def validate_geometry_types(self):
        """Check geometry types"""
        print("\n" + "="*60)
        print("📐 GEOMETRY TYPE VALIDATION")
        print("="*60)

        query = """
        SELECT
            'states' AS TableName,
            Shape.STGeometryType() AS GeometryType,
            COUNT(*) AS Count
        FROM states
        WHERE Shape IS NOT NULL
        GROUP BY Shape.STGeometryType()

        UNION ALL

        SELECT
            'fedlandp' AS TableName,
            Shape.STGeometryType() AS GeometryType,
            COUNT(*) AS Count
        FROM fedlandp
        WHERE Shape IS NOT NULL
        GROUP BY Shape.STGeometryType()

        UNION ALL

        SELECT
            'park_dtl' AS TableName,
            Shape.STGeometryType() AS GeometryType,
            COUNT(*) AS Count
        FROM park_dtl
        WHERE Shape IS NOT NULL
        GROUP BY Shape.STGeometryType()

        UNION ALL

        SELECT
            'dtl_st' AS TableName,
            Shape.STGeometryType() AS GeometryType,
            COUNT(*) AS Count
        FROM dtl_st
        WHERE Shape IS NOT NULL
        GROUP BY Shape.STGeometryType()

        ORDER BY TableName, GeometryType
        """

        df = self.run_query(query, "Checking geometry types")
        if df is not None:
            print("\n📐 Geometry Types:")
            current_table = None
            for _, row in df.iterrows():
                table = row['TableName']
                geom_type = row['GeometryType']
                count = row['Count']

                if table != current_table:
                    print(f"\n   📋 {table}:")
                    current_table = table

                print(f"      {geom_type}: {count:,}")

        return df

    def validate_spatial_indexes(self):
        """Check spatial indexes"""
        print("\n" + "="*60)
        print("🗂️ SPATIAL INDEX VALIDATION")
        print("="*60)

        query = """
        SELECT
            t.name AS TableName,
            i.name AS IndexName,
            i.type_desc AS IndexType,
            CASE WHEN i.type = 4 THEN 'SPATIAL' ELSE 'OTHER' END AS IndexCategory
        FROM sys.tables t
        INNER JOIN sys.indexes i ON t.object_id = i.object_id
        WHERE t.name IN ('states', 'fedlandp', 'park_dtl', 'dtl_st')
        ORDER BY t.name, i.type
        """

        df = self.run_query(query, "Checking spatial indexes")
        if df is not None:
            print("\n🗂️ Index Summary:")
            spatial_indexes = df[df['IndexCategory'] == 'SPATIAL']

            for table in ['states', 'fedlandp', 'park_dtl', 'dtl_st']:
                table_indexes = spatial_indexes[spatial_indexes['TableName'] == table]
                if len(table_indexes) > 0:
                    print(f"   ✅ {table}: {len(table_indexes)} spatial index(es)")
                    for _, idx in table_indexes.iterrows():
                        print(f"      - {idx['IndexName']}")
                else:
                    print(f"   ❌ {table}: No spatial indexes")

        return df

    def test_spatial_operations(self):
        """Test basic spatial operations"""
        print("\n" + "="*60)
        print("🧪 SPATIAL OPERATIONS TEST")
        print("="*60)

        # Test 1: Basic spatial queries
        query1 = """
        SELECT TOP 1
            'Area calculation test' AS Test,
            CAST(Shape.STArea() AS FLOAT) AS Area,
            Shape.STGeometryType() AS GeomType
        FROM states
        WHERE Shape IS NOT NULL
        ORDER BY Shape.STArea() DESC
        """

        df1 = self.run_query(query1, "Testing area calculations")
        if df1 is not None and len(df1) > 0:
            area = df1.iloc[0]['Area']
            geom_type = df1.iloc[0]['GeomType']
            print(f"   ✅ Largest state area: {area:,.2f} sq meters ({geom_type})")

        # Test 2: Count geometries by SRID
        query2 = """
        SELECT
            'states' AS TableName,
            Shape.STSrid AS SRID,
            COUNT(*) AS Count
        FROM states
        WHERE Shape IS NOT NULL
        GROUP BY Shape.STSrid
        """

        df2 = self.run_query(query2, "Testing coordinate reference systems")
        if df2 is not None:
            print("\n   🌐 Coordinate Systems:")
            for _, row in df2.iterrows():
                srid = row['SRID']
                count = row['Count']
                status = "✅" if srid == 4326 else "⚠️"
                print(f"      {status} SRID {srid}: {count:,} records")

        return df1, df2

    def generate_validation_report(self):
        """Generate comprehensive validation report"""
        print("\n" + "="*80)
        print("📊 FGDB TO SQL SERVER CONVERSION VALIDATION REPORT")
        print("="*80)

        if not self.connect():
            return False

        try:
            # Run all validations
            structure_df = self.validate_basic_structure()
            geometry_df = self.validate_geometry()
            geom_type_df = self.validate_geometry_types()
            index_df = self.validate_spatial_indexes()
            spatial_test = self.test_spatial_operations()

            # Summary
            print("\n" + "="*60)
            print("🎯 VALIDATION SUMMARY")
            print("="*60)

            total_records = structure_df['RecordCount'].sum() if structure_df is not None else 0
            print(f"📈 Total Records Converted: {total_records:,}")

            if geometry_df is not None:
                total_valid_geom = geometry_df['ValidGeometries'].sum()
                total_invalid_geom = geometry_df['InvalidGeometries'].sum()
                print(f"🗺️ Valid Geometries: {total_valid_geom:,}")
                print(f"⚠️ Invalid Geometries: {total_invalid_geom:,}")

                if total_invalid_geom == 0:
                    print("✅ Geometry Validation: PASSED")
                else:
                    print("❌ Geometry Validation: ISSUES FOUND")

            spatial_indexes = index_df[index_df['IndexCategory'] == 'SPATIAL'] if index_df is not None else pd.DataFrame()
            print(f"🗂️ Spatial Indexes: {len(spatial_indexes)}/4 tables")

            # Overall assessment
            print(f"\n🏆 OVERALL ASSESSMENT:")

            issues = []
            if total_records != 30096:
                issues.append("Record count mismatch")
            if geometry_df is not None and geometry_df['InvalidGeometries'].sum() > 0:
                issues.append("Invalid geometries found")
            if len(spatial_indexes) < 4:
                issues.append("Missing spatial indexes")

            if len(issues) == 0:
                print("✅ CONVERSION SUCCESSFUL - No issues found")
                print("🎉 All data converted accurately with proper spatial support")
            else:
                print(f"⚠️ CONVERSION COMPLETED WITH ISSUES:")
                for issue in issues:
                    print(f"   - {issue}")

            return len(issues) == 0

        except Exception as e:
            print(f"❌ Validation failed: {e}")
            return False
        finally:
            if self.conn:
                self.conn.close()

def main():
    """Run the simplified SQL validation"""

    SQL_CONFIG = {
        'server': '100.103.17.32,1433',
        'database': 'SpatialTest',
        'username': 'dbeaver',
        'password': 'dbeaver'
    }

    validator = SimpleSQLValidator(SQL_CONFIG)
    success = validator.generate_validation_report()

    return success

if __name__ == "__main__":
    validation_success = main()

In [ ]:
"""
Simple SQL-Based Validation for FGDB Conversion
Uses only SQL Server queries to validate the conversion results
"""

import pyodbc
import pandas as pd

class SimpleSQLValidator:
    def __init__(self, sql_config):
        self.sql_config = sql_config
        self.conn = None

    def connect(self):
        """Create SQL Server connection"""
        try:
            conn_str = (
                f"DRIVER={{ODBC Driver 17 for SQL Server}};"
                f"SERVER={self.sql_config['server']};"
                f"DATABASE={self.sql_config['database']};"
                f"UID={self.sql_config['username']};"
                f"PWD={self.sql_config['password']};"
            )
            self.conn = pyodbc.connect(conn_str)
            print("✅ Connected to SQL Server")
            return True
        except Exception as e:
            print(f"❌ Connection failed: {e}")
            return False

    def run_query(self, query, description):
        """Execute a query and return results"""
        try:
            print(f"\n🔍 {description}")
            df = pd.read_sql(query, self.conn)
            return df
        except Exception as e:
            print(f"❌ Query failed: {e}")
            return None

    def validate_basic_structure(self):
        """Validate basic table structure and record counts"""
        print("\n" + "="*60)
        print("📋 BASIC STRUCTURE VALIDATION")
        print("="*60)

        # Check if tables exist and get record counts
        query = """
        SELECT
            t.TABLE_NAME,
            (SELECT COUNT(*) FROM states WHERE t.TABLE_NAME = 'states') AS RecordCount
        FROM INFORMATION_SCHEMA.TABLES t
        WHERE t.TABLE_NAME = 'states'

        UNION ALL

        SELECT
            t.TABLE_NAME,
            (SELECT COUNT(*) FROM fedlandp WHERE t.TABLE_NAME = 'fedlandp') AS RecordCount
        FROM INFORMATION_SCHEMA.TABLES t
        WHERE t.TABLE_NAME = 'fedlandp'

        UNION ALL

        SELECT
            t.TABLE_NAME,
            (SELECT COUNT(*) FROM park_dtl WHERE t.TABLE_NAME = 'park_dtl') AS RecordCount
        FROM INFORMATION_SCHEMA.TABLES t
        WHERE t.TABLE_NAME = 'park_dtl'

        UNION ALL

        SELECT
            t.TABLE_NAME,
            (SELECT COUNT(*) FROM dtl_st WHERE t.TABLE_NAME = 'dtl_st') AS RecordCount
        FROM INFORMATION_SCHEMA.TABLES t
        WHERE t.TABLE_NAME = 'dtl_st'

        ORDER BY TABLE_NAME
        """

        df = self.run_query(query, "Checking table existence and record counts")
        if df is not None:
            print("\n📊 Table Summary:")
            for _, row in df.iterrows():
                print(f"   ✅ {row['TABLE_NAME']}: {row['RecordCount']:,} records")

            total_records = df['RecordCount'].sum()
            print(f"\n📈 Total Records: {total_records:,}")

            # Expected counts from our conversion
            expected = {'states': 51, 'fedlandp': 3687, 'park_dtl': 26307, 'dtl_st': 51}
            print(f"\n🎯 Expected vs Actual:")
            for _, row in df.iterrows():
                table = row['TABLE_NAME']
                actual = row['RecordCount']
                exp = expected.get(table, 0)
                status = "✅" if actual == exp else "❌"
                print(f"   {status} {table}: {actual:,} (expected {exp:,})")

        return df

    def validate_geometry(self):
        """Validate geometry data quality"""
        print("\n" + "="*60)
        print("🗺️ GEOMETRY VALIDATION")
        print("="*60)

        query = """
        SELECT
            'states' AS TableName,
            COUNT(*) AS TotalRecords,
            COUNT(Shape) AS RecordsWithGeometry,
            SUM(CASE WHEN Shape IS NOT NULL AND Shape.STIsValid() = 1 THEN 1 ELSE 0 END) AS ValidGeometries,
            SUM(CASE WHEN Shape IS NOT NULL AND Shape.STIsValid() = 0 THEN 1 ELSE 0 END) AS InvalidGeometries,
            AVG(CAST(Shape.STArea() AS FLOAT)) AS AvgArea
        FROM states

        UNION ALL

        SELECT
            'fedlandp' AS TableName,
            COUNT(*) AS TotalRecords,
            COUNT(Shape) AS RecordsWithGeometry,
            SUM(CASE WHEN Shape IS NOT NULL AND Shape.STIsValid() = 1 THEN 1 ELSE 0 END) AS ValidGeometries,
            SUM(CASE WHEN Shape IS NOT NULL AND Shape.STIsValid() = 0 THEN 1 ELSE 0 END) AS InvalidGeometries,
            AVG(CAST(Shape.STArea() AS FLOAT)) AS AvgArea
        FROM fedlandp

        UNION ALL

        SELECT
            'park_dtl' AS TableName,
            COUNT(*) AS TotalRecords,
            COUNT(Shape) AS RecordsWithGeometry,
            SUM(CASE WHEN Shape IS NOT NULL AND Shape.STIsValid() = 1 THEN 1 ELSE 0 END) AS ValidGeometries,
            SUM(CASE WHEN Shape IS NOT NULL AND Shape.STIsValid() = 0 THEN 1 ELSE 0 END) AS InvalidGeometries,
            AVG(CAST(Shape.STArea() AS FLOAT)) AS AvgArea
        FROM park_dtl

        UNION ALL

        SELECT
            'dtl_st' AS TableName,
            COUNT(*) AS TotalRecords,
            COUNT(Shape) AS RecordsWithGeometry,
            SUM(CASE WHEN Shape IS NOT NULL AND Shape.STIsValid() = 1 THEN 1 ELSE 0 END) AS ValidGeometries,
            SUM(CASE WHEN Shape IS NOT NULL AND Shape.STIsValid() = 0 THEN 1 ELSE 0 END) AS InvalidGeometries,
            AVG(CAST(Shape.STArea() AS FLOAT)) AS AvgArea
        FROM dtl_st

        ORDER BY TableName
        """

        df = self.run_query(query, "Validating geometry data quality")
        if df is not None:
            print("\n🗺️ Geometry Summary:")
            for _, row in df.iterrows():
                table = row['TableName']
                total = row['TotalRecords']
                with_geom = row['RecordsWithGeometry']
                valid = row['ValidGeometries']
                invalid = row['InvalidGeometries']
                avg_area = row['AvgArea']

                print(f"\n   📋 {table}:")
                print(f"      Total Records: {total:,}")
                print(f"      With Geometry: {with_geom:,}")
                print(f"      Valid Geometry: {valid:,}")
                print(f"      Invalid Geometry: {invalid:,}")
                print(f"      Avg Area: {avg_area:,.2f} sq meters")

                # Validation status
                if total == with_geom == valid and invalid == 0:
                    print(f"      Status: ✅ PERFECT")
                elif invalid == 0:
                    print(f"      Status: ✅ GOOD")
                else:
                    print(f"      Status: ⚠️ HAS ISSUES")

        return df

    def validate_geometry_types(self):
        """Check geometry types"""
        print("\n" + "="*60)
        print("📐 GEOMETRY TYPE VALIDATION")
        print("="*60)

        query = """
        SELECT
            'states' AS TableName,
            Shape.STGeometryType() AS GeometryType,
            COUNT(*) AS Count
        FROM states
        WHERE Shape IS NOT NULL
        GROUP BY Shape.STGeometryType()

        UNION ALL

        SELECT
            'fedlandp' AS TableName,
            Shape.STGeometryType() AS GeometryType,
            COUNT(*) AS Count
        FROM fedlandp
        WHERE Shape IS NOT NULL
        GROUP BY Shape.STGeometryType()

        UNION ALL

        SELECT
            'park_dtl' AS TableName,
            Shape.STGeometryType() AS GeometryType,
            COUNT(*) AS Count
        FROM park_dtl
        WHERE Shape IS NOT NULL
        GROUP BY Shape.STGeometryType()

        UNION ALL

        SELECT
            'dtl_st' AS TableName,
            Shape.STGeometryType() AS GeometryType,
            COUNT(*) AS Count
        FROM dtl_st
        WHERE Shape IS NOT NULL
        GROUP BY Shape.STGeometryType()

        ORDER BY TableName, GeometryType
        """

        df = self.run_query(query, "Checking geometry types")
        if df is not None:
            print("\n📐 Geometry Types:")
            current_table = None
            for _, row in df.iterrows():
                table = row['TableName']
                geom_type = row['GeometryType']
                count = row['Count']

                if table != current_table:
                    print(f"\n   📋 {table}:")
                    current_table = table

                print(f"      {geom_type}: {count:,}")

        return df

    def validate_spatial_indexes(self):
        """Check spatial indexes"""
        print("\n" + "="*60)
        print("🗂️ SPATIAL INDEX VALIDATION")
        print("="*60)

        query = """
        SELECT
            t.name AS TableName,
            i.name AS IndexName,
            i.type_desc AS IndexType,
            CASE WHEN i.type = 4 THEN 'SPATIAL' ELSE 'OTHER' END AS IndexCategory
        FROM sys.tables t
        INNER JOIN sys.indexes i ON t.object_id = i.object_id
        WHERE t.name IN ('states', 'fedlandp', 'park_dtl', 'dtl_st')
        ORDER BY t.name, i.type
        """

        df = self.run_query(query, "Checking spatial indexes")
        if df is not None:
            print("\n🗂️ Index Summary:")
            spatial_indexes = df[df['IndexCategory'] == 'SPATIAL']

            for table in ['states', 'fedlandp', 'park_dtl', 'dtl_st']:
                table_indexes = spatial_indexes[spatial_indexes['TableName'] == table]
                if len(table_indexes) > 0:
                    print(f"   ✅ {table}: {len(table_indexes)} spatial index(es)")
                    for _, idx in table_indexes.iterrows():
                        print(f"      - {idx['IndexName']}")
                else:
                    print(f"   ❌ {table}: No spatial indexes")

        return df

    def test_spatial_operations(self):
        """Test basic spatial operations"""
        print("\n" + "="*60)
        print("🧪 SPATIAL OPERATIONS TEST")
        print("="*60)

        # Test 1: Basic spatial queries
        query1 = """
        SELECT TOP 1
            'Area calculation test' AS Test,
            CAST(Shape.STArea() AS FLOAT) AS Area,
            Shape.STGeometryType() AS GeomType
        FROM states
        WHERE Shape IS NOT NULL
        ORDER BY Shape.STArea() DESC
        """

        df1 = self.run_query(query1, "Testing area calculations")
        if df1 is not None and len(df1) > 0:
            area = df1.iloc[0]['Area']
            geom_type = df1.iloc[0]['GeomType']
            print(f"   ✅ Largest state area: {area:,.2f} sq meters ({geom_type})")

        # Test 2: Count geometries by SRID
        query2 = """
        SELECT
            'states' AS TableName,
            Shape.STSrid AS SRID,
            COUNT(*) AS Count
        FROM states
        WHERE Shape IS NOT NULL
        GROUP BY Shape.STSrid
        """

        df2 = self.run_query(query2, "Testing coordinate reference systems")
        if df2 is not None:
            print("\n   🌐 Coordinate Systems:")
            for _, row in df2.iterrows():
                srid = row['SRID']
                count = row['Count']
                status = "✅" if srid == 4326 else "⚠️"
                print(f"      {status} SRID {srid}: {count:,} records")

        return df1, df2

    def generate_validation_report(self):
        """Generate comprehensive validation report"""
        print("\n" + "="*80)
        print("📊 FGDB TO SQL SERVER CONVERSION VALIDATION REPORT")
        print("="*80)

        if not self.connect():
            return False

        try:
            # Run all validations
            structure_df = self.validate_basic_structure()
            geometry_df = self.validate_geometry()
            geom_type_df = self.validate_geometry_types()
            index_df = self.validate_spatial_indexes()
            spatial_test = self.test_spatial_operations()

            # Summary
            print("\n" + "="*60)
            print("🎯 VALIDATION SUMMARY")
            print("="*60)

            total_records = structure_df['RecordCount'].sum() if structure_df is not None else 0
            print(f"📈 Total Records Converted: {total_records:,}")

            if geometry_df is not None:
                total_valid_geom = geometry_df['ValidGeometries'].sum()
                total_invalid_geom = geometry_df['InvalidGeometries'].sum()
                print(f"🗺️ Valid Geometries: {total_valid_geom:,}")
                print(f"⚠️ Invalid Geometries: {total_invalid_geom:,}")

                if total_invalid_geom == 0:
                    print("✅ Geometry Validation: PASSED")
                else:
                    print("❌ Geometry Validation: ISSUES FOUND")

            spatial_indexes = index_df[index_df['IndexCategory'] == 'SPATIAL'] if index_df is not None else pd.DataFrame()
            print(f"🗂️ Spatial Indexes: {len(spatial_indexes)}/4 tables")

            # Overall assessment
            print(f"\n🏆 OVERALL ASSESSMENT:")

            issues = []
            if total_records != 30096:
                issues.append("Record count mismatch")
            if geometry_df is not None and geometry_df['InvalidGeometries'].sum() > 0:
                issues.append("Invalid geometries found")
            if len(spatial_indexes) < 4:
                issues.append("Missing spatial indexes")

            if len(issues) == 0:
                print("✅ CONVERSION SUCCESSFUL - No issues found")
                print("🎉 All data converted accurately with proper spatial support")
            else:
                print(f"⚠️ CONVERSION COMPLETED WITH ISSUES:")
                for issue in issues:
                    print(f"   - {issue}")

            return len(issues) == 0

        except Exception as e:
            print(f"❌ Validation failed: {e}")
            return False
        finally:
            if self.conn:
                self.conn.close()

def main():
    """Run the simplified SQL validation"""

    SQL_CONFIG = {
        'server': '100.103.17.32,1433',
        'database': 'SpatialTest',
        'username': 'dbeaver',
        'password': 'dbeaver'
    }

    validator = SimpleSQLValidator(SQL_CONFIG)
    success = validator.generate_validation_report()

    return success

if __name__ == "__main__":
    validation_success = main()